Generando modelos de manera estática para asegurar reproducibilidad

In [1]:
model_patch_size = 8
verbose          = 0
reset_kernel     = False

In [2]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<froze


Octave is ready <oct2py.core.Oct2Py object at 0x7ff6560aa0e0>

















In [3]:
import torch
torch.cuda.set_device(0)
#torch.cuda.set_device(1)

# Generando los modelos para khol's

In [4]:
# User and project
entity = os.environ.get("WANDB_ENTITY")
project = os.environ.get("WANDB_PROJECT")
folder = entity+'/'+project+'/'

# Dataset
dataset = 'gtrends_kohls'
dataset_version = 'v2'
artifact_dataset = folder + dataset + ':' + dataset_version

# Models
model_family = 'zeroshot-moment'
task = 'embedding'
artifact_small_name = folder + model_family + '-small-' + task + ':v0'
artifact_base_name  =  folder + model_family + '-base-' + task + ':v0'
artifact_large_name = folder + model_family + '-large-' + task + ':v0'

Download kohls' dataset

In [6]:
print("Getting dataset artifact: ", artifact_dataset)
df_artifact = wandb_api.artifact(artifact_dataset, type = 'dataset')
df = df_artifact.to_df()

Getting dataset artifact:  mi-santamaria/deepvats/gtrends_kohls:v2


wandb:   1 of 1 files downloaded.  


In [7]:
df.head()

,volume
2004-01-01,0.010417
2004-01-08,0.010417
2004-01-15,0.010417
2004-01-22,0.000000
2004-01-29,0.000000


In [23]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": 16,
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": None,
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": 1e-6,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": [12],
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 0,
    "print_to_path": True,
    "print_both": True,
    "print_path": "./generate-paper-models.log",
    "print_mode": "a",
    "use_moment_masks": True,
    "mask_stateful": False,
    "mask_future": True,
    "mask_sync": False,
    "analysis_mode": 'online',
    "use_wandb": True,
    "norm_by_sample": False,
    "norm_use_single_batch": False,
    "show_plot": True,
    "metrics": None,
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None,
    "criterion": torch.nn.MSELoss(),
    "training_percent": 0.25,
    "validation_percent": 0.3,
    "num_epochs" : 1, 
    "n_window_sizes": 1,
    "register_errors": False,
    "save_best_or_last": True
}

In [12]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="_paper_022025-generate_models"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname="paper-foundational-generate_models"
print("runname: "+runname)

runname: paper-foundational-generate_models


In [14]:
run = wandb.init(
    entity      = entity,
    project     = project,
    group       = 'foundational-model',
    job_type    = 'moment',
    mode        = 'online',
    anonymous   = 'never',
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


In [15]:
artifacts_gettr = run.use_artifact

In [16]:
print("Getting small artifact: ", artifact_small_name)
artifact_small = wandb_api.artifact(artifact_small_name, type='learner')
print(artifact_small.name)
moment_small = artifact_small.to_obj()

Getting small artifact:  mi-santamaria/deepvats/zeroshot-moment-small-embedding:v0
zeroshot-moment-small-embedding:v0


wandb: Downloading large artifact zeroshot-moment-small-embedding:v0, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


In [17]:
# Epochs: 1
# Dataset percent: 0.25
# Only 1 window size

## Small - 25%

In [20]:
from copy import deepcopy

In [42]:
tuned_model_small_25 = deepcopy(moment_small)
result_small_25 = fine_tune(
    enc_learn           = tuned_model_small_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/9 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 11% 1/9 [00:01<00:10,  1.37s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 78% 7/9 [00:03<00:00,  2.48it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 89% 8/9 [00:03<00:00,  2.50it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 9/9 [00:04<00:00,  2.10it/s]


In [26]:
tuned_model = deepcopy(result[7])
tuned_model.__class__

momentfm.models.moment.MOMENTPipeline

In [50]:
time_small_25 = result_small_25[4]
print("time: ", time_small_25)
loss_pre_small_25 = result_small_25[1]['loss'][0]
print("loss pre: ", loss_pre_small_25)
loss_post_small_25 = result_small_25[2]['loss'][0]
print("loss_post: ", loss_post_small_25)
improvement_small_25 = (loss_pre_small_25-loss_post_small_25)*100/loss_pre_small_25
print(improvement_small_25)

time:  1.6744823455810547
loss pre:  0.003403914258039246
loss_post:  0.0034287193977636183
-0.7287239878557014


In [48]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_small_25,
        f'moment-small-kohls-25',
        type = 'learner'
    )
)

## Small-75%

In [88]:
tuned_model_small_75 = deepcopy(moment_small)
result_small_75 = fine_tune(
    enc_learn           = tuned_model_small_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:34,  1.45s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.10it/s]


In [89]:
tuned_model_small_75 = deepcopy(result_small_75[7])
print(tuned_model_small_75.__class__)
time_small_75 = result_small_75[4]
print(time_small_75)
loss_pre_small_75 = result_small_75[1]['loss'][0]
print(loss_pre_small_75)
loss_post_small_75 = result_small_75[2]['loss'][0]
print(loss_post_small_75)
improvement_small_75 = (loss_pre_small_75-loss_post_small_75)*100/loss_pre_small_75
print(improvement_small_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
5.258074045181274
11.070083484649658
11.0995304107666
-0.26600455324276306


In [90]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_small_75,
        f'moment-small-kohls-75',
        type = 'learner'
    )
)

## Base 25%

In [38]:
print("Getting base artifact: ", artifact_base_name)
artifact_base = wandb_api.artifact(artifact_base_name, type='learner')
print(artifact_base.name)
moment_base = artifact_base.to_obj()

Getting base artifact:  mi-santamaria/deepvats/zeroshot-moment-base-embedding:v0
zeroshot-moment-base-embedding:v0


wandb: Downloading large artifact zeroshot-moment-base-embedding:v0, 432.97MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.3


In [55]:
tuned_model_base_25 = deepcopy(moment_base)
result_base_25 = fine_tune(
    enc_learn           = tuned_model_base_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log



  0% 0/9 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 11% 1/9 [00:00<00:02,  2.70it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...


 78% 7/9 [00:03<00:00,  2.43it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 89% 8/9 [00:03<00:00,  2.50it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



100% 9/9 [00:04<00:00,  2.08it/s]


In [56]:
tuned_model_base_25 = deepcopy(result_base_25[7])
print(tuned_model.__class__)
time_base_25 = result_base_25[4]
print("Train time: ", time_base_25)
loss_pre_base_25 = result_base_25[1]['loss'][0]
print("Loss pre:", loss_pre_base_25)
loss_post_base_25 = result_base_25[2]['loss'][0]
print("Loss post:", loss_post_base_25)
improvement_base_25 = (loss_pre_base_25-loss_post_base_25)*100/loss_pre_base_25
print("Improvement: ", improvement_base_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.6916606426239014
Loss pre: 0.0033898308595073307
Loss post: 0.0032449673647837094
Improvement:  4.2734726518088095


In [58]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_base_25,
        f'moment-base-kohls-25',
        type = 'learner'
    )
)

## Base 75%

In [60]:
tuned_model_base_75 = deepcopy(moment_base)
result_base_75 = fine_tune(
    enc_learn           = tuned_model_base_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log



  0% 0/9 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 11% 1/9 [00:01<00:11,  1.38s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...


 78% 7/9 [00:03<00:00,  2.40it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 89% 8/9 [00:04<00:00,  2.42it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



100% 9/9 [00:04<00:00,  2.01it/s]


In [76]:
tuned_model_base_75 = deepcopy(result_base_75[7])
print(tuned_model.__class__)
time_base_75 = result_base_75[4]
print("Train time: ", time_base_75)
loss_pre_base_75 = result_base_75[1]['loss'][0]
print("Loss pre:", loss_pre_base_75)
loss_post_base_75 = result_base_75[2]['loss'][0]
print("Loss post:", loss_post_base_25)
improvement_base_75 = (loss_pre_base_75-loss_post_base_75)*100/loss_pre_base_75
print("Improvement: ", improvement_base_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.748645544052124
Loss pre: 0.0034909040324338195
Loss post: 0.0032449673647837094
Improvement:  10.088424451112202


In [62]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_base_75,
        f'moment-base-kohls-75',
        type = 'learner'
    )
)

## Large 25%

In [59]:
print("Getting large artifact: ", artifact_large_name)
artifact_large = wandb_api.artifact(artifact_large_name, type='learner')
print(artifact_large.name)
moment_large = artifact_large.to_obj()

Getting large artifact:  mi-santamaria/deepvats/zeroshot-moment-large-embedding:v0
zeroshot-moment-large-embedding:v0


wandb: Downloading large artifact zeroshot-moment-large-embedding:v0, 1321.42MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:4.4


In [63]:
tuned_model_large_25 = deepcopy(moment_large)
result_large_25 = fine_tune(
    enc_learn           = tuned_model_large_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log



  0% 0/9 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 11% 1/9 [00:01<00:11,  1.40s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...


 78% 7/9 [00:04<00:00,  2.11it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 89% 8/9 [00:04<00:00,  2.20it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



100% 9/9 [00:04<00:00,  1.88it/s]


In [68]:
tuned_model_large_25 = deepcopy(result_large_25[7])
print(tuned_model_large_25.__class__)
time_large_25 = result_large_25[4]
print("Train time: ", time_large_25)
loss_pre_large_25 = result_large_25[1]['loss'][0]
print("Loss pre:", loss_pre_large_25)
loss_post_large_25 = result_large_25[2]['loss'][0]
print("Loss post:", loss_post_large_25)
improvement_base_25 = (loss_pre_large_25-loss_post_large_25)*100/loss_pre_large_25
print("Improvement: ", improvement_base_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.9187376499176025
Loss pre: 0.0018025103911188329
Loss post: 0.0019364783236394739
Improvement:  -7.432297377075648


In [ ]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_large_25,
        f'moment-large-kohls-25',
        type = 'learner'
    )
)

## Large 75%

In [65]:
tuned_model_large_75 = deepcopy(moment_large)
result_large_75 = fine_tune(
    enc_learn           = tuned_model_large_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log



  0% 0/9 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 11% 1/9 [00:01<00:11,  1.40s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...


 78% 7/9 [00:03<00:00,  2.28it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



 89% 8/9 [00:04<00:00,  2.21it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log



100% 9/9 [00:04<00:00,  1.94it/s]


In [74]:
tuned_model_large_75 = deepcopy(result_large_75[7])
print(tuned_model_large_75.__class__)
time_large_75 = result_large_75[4]
print("Train time: ", time_large_75)
loss_pre_large_75 = result_large_75[1]['loss'][0]
print("Loss pre:", loss_pre_large_75)
loss_post_large_75 = result_large_75[2]['loss'][0]
print("Loss post:", loss_post_large_75)
improvement_large_75 = (loss_pre_large_75-loss_post_large_75)*100/loss_pre_large_75
print("Improvement: ", improvement_large_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.8700814247131348
Loss pre: 0.003920885999428315
Loss post: 0.0038622432491845554
Improvement:  1.4956504793128422


In [75]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_large_75,
        f'moment-large-kohls-75',
        type = 'learner'
    )
)

## Summary khol's

In [77]:
print("--- Small 25 ---")
time_small_25 = result_small_25[4]
print("time: ", time_small_25)
loss_pre_small_25 = result_small_25[1]['loss'][0]
print("loss pre: ", loss_pre_small_25)
loss_post_small_25 = result_small_25[2]['loss'][0]
print("loss_post: ", loss_post_small_25)
improvement_small_25 = (loss_pre_small_25-loss_post_small_25)*100/loss_pre_small_25
print(improvement_small_25)
print("--- Small 75 ---")
print(tuned_model_small_75.__class__)
time_small_75 = result_small_75[4]
print(time_small_75)
loss_pre_small_75 = result_small_75[1]['loss'][0]
print(loss_pre_small_75)
loss_post_small_75 = result_small_75[2]['loss'][0]
print(loss_post_small_75)
improvement_small_75 = (loss_pre_small_75-loss_post_small_75)*100/loss_pre_small_75
print(improvement_small_75)
print("--- Base 25 ---")
print(tuned_model.__class__)
time_base_25 = result_base_25[4]
print("Train time: ", time_base_25)
loss_pre_base_25 = result_base_25[1]['loss'][0]
print("Loss pre:", loss_pre_base_25)
loss_post_base_25 = result_base_25[2]['loss'][0]
print("Loss post:", loss_post_base_25)
improvement_base_25 = (loss_pre_base_25-loss_post_base_25)*100/loss_pre_base_25
print("Improvement: ", improvement_base_25)
print("--- Base 75 ---") 
print(tuned_model.__class__)
time_base_75 = result_base_75[4]
print("Train time: ", time_base_75)
loss_pre_base_75 = result_base_75[1]['loss'][0]
print("Loss pre:", loss_pre_base_75)
loss_post_base_75 = result_base_75[2]['loss'][0]
print("Loss post:", loss_post_base_75)
improvement_base_75 = (loss_pre_base_75-loss_post_base_75)*100/loss_pre_base_75
print("Improvement: ", improvement_base_75)
print("--- Large 25 ---") 
print(tuned_model_large_25.__class__)
time_large_25 = result_large_25[4]
print("Train time: ", time_large_25)
loss_pre_large_25 = result_large_25[1]['loss'][0]
print("Loss pre:", loss_pre_large_25)
loss_post_large_25 = result_large_25[2]['loss'][0]
print("Loss post:", loss_post_large_25)
improvement_base_25 = (loss_pre_large_25-loss_post_large_25)*100/loss_pre_large_25
print("Improvement: ", improvement_base_25)
print("--- Large 75 ---")
tuned_model_large_75 = deepcopy(result_large_75[7])
print(tuned_model_large_75.__class__)
time_large_75 = result_large_75[4]
print("Train time: ", time_large_75)
loss_pre_large_75 = result_large_75[1]['loss'][0]
print("Loss pre:", loss_pre_large_75)
loss_post_large_75 = result_large_75[2]['loss'][0]
print("Loss post:", loss_post_large_75)
improvement_large_75 = (loss_pre_large_75-loss_post_large_75)*100/loss_pre_large_75
print("Improvement: ", improvement_large_75)

--- Small 25 ---
time:  1.6744823455810547
loss pre:  0.003403914258039246
loss_post:  0.0034287193977636183
-0.7287239878557014
--- Small 75 ---
<class 'momentfm.models.moment.MOMENTPipeline'>
1.6892859935760498
0.0038138289431420467
0.003971449348480544
-4.132865099309904
--- Base 25 ---
<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.6916606426239014
Loss pre: 0.0033898308595073307
Loss post: 0.0032449673647837094
Improvement:  4.2734726518088095
--- Base 75 ---
<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.748645544052124
Loss pre: 0.0034909040324338195
Loss post: 0.003138726816460904
Improvement:  10.088424451112202
--- Large 25 ---
<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.9187376499176025
Loss pre: 0.0018025103911188329
Loss post: 0.0019364783236394739
Improvement:  -7.432297377075648
--- Large 75 ---
<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  1.8700814247131348
Loss pre: 0.003920885999428315
Loss post: 0.003

# Generando los modelos para S1

In [82]:
# Dataset
dataset = 'S1'
dataset_version = 'v8'
artifact_dataset = folder + dataset + ':' + dataset_version
print("Getting dataset artifact: ", artifact_dataset)
s1_artifact = wandb_api.artifact(artifact_dataset, type = 'dataset')
s1 = s1_artifact.to_df()
print(s1.shape)
s1.head()

Getting dataset artifact:  mi-santamaria/deepvats/S1:v8


wandb:   1 of 1 files downloaded.  


(2688, 1)


,Seasonal time series w/ noise
2022-03-01 00:00:00,2.210769
2022-03-01 00:15:00,3.480337
2022-03-01 00:30:00,3.862855
2022-03-01 00:45:00,4.758258
2022-03-01 01:00:00,3.205716


In [83]:
common_args = {
    "X": s1,
    "stride": 1,
    "batch_size": 32,
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": None,
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": 1e-6,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": [72],
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 0,
    "print_to_path": True,
    "print_both": True,
    "print_path": "./generate-paper-models.log",
    "print_mode": "a",
    "use_moment_masks": True,
    "mask_stateful": True,
    "mask_future": False,
    "mask_sync": False,
    "analysis_mode": 'online',
    "use_wandb": True,
    "norm_by_sample": False,
    "norm_use_single_batch": False,
    "show_plot": True,
    "metrics": None,
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None,
    "criterion": torch.nn.MSELoss(),
    "training_percent": 0.25,
    "validation_percent": 0.3,
    "num_epochs" : 1, 
    "n_window_sizes": 1,
    "register_errors": False,
    "save_best_or_last": True
}

## Small 25

In [85]:
s1_tuned_model_small_25 = deepcopy(moment_small)
s1_result_small_25 = fine_tune(
    enc_learn           = s1_tuned_model_small_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.46s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:10<00:00,  2.37it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.36it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.11it/s]


In [86]:
s1_tuned_model_small_25 = deepcopy(result[7])
s1_tuned_model_small_25.__class__
s1_time_small_25 = s1_result_small_25[4]
print("time: ", s1_time_small_25)
s1_loss_pre_small_25 = s1_result_small_25[1]['loss'][0]
print("loss pre: ", s1_loss_pre_small_25)
s1_loss_post_small_25 = s1_result_small_25[2]['loss'][0]
print("loss_post: ", s1_loss_post_small_25)
s1_improvement_small_25 = (s1_loss_pre_small_25-s1_loss_post_small_25)*100/s1_loss_pre_small_25
print(s1_improvement_small_25)

time:  5.177350044250488
loss pre:  10.927521171569824
loss_post:  10.954853057861328
-0.25011972854936104


In [87]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        s1_tuned_model_small_25,
        f'moment-small-s1-25',
        type = 'learner'
    )
)

## Small 75

In [91]:
s1_tuned_model_small_75 = deepcopy(moment_small)
s1_result_small_75 = fine_tune(
    enc_learn           = s1_tuned_model_small_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:34,  1.45s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:10<00:00,  2.39it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.40it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.16it/s]


In [92]:
s1_tuned_model_small_75 = deepcopy(s1_result_small_75[7])
print(s1_tuned_model_small_75.__class__)
s1_time_small_75 = s1_result_small_75[4]
print(s1_time_small_75)
s1_loss_pre_small_75 = s1_result_small_75[1]['loss'][0]
print(s1_loss_pre_small_75)
s1_loss_post_small_75 = s1_result_small_75[2]['loss'][0]
print(s1_loss_post_small_75)
s1_improvement_small_75 = (s1_loss_pre_small_75-s1_loss_post_small_75)*100/s1_loss_pre_small_75
print(s1_improvement_small_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
5.014077663421631
11.1762308883667
11.139793300628662
0.3260275141234326


In [93]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        s1_tuned_model_small_75,
        f'moment-small-s1-75',
        type = 'learner'
    )
)

## Base 25

In [94]:
s1_tuned_model_base_25 = deepcopy(moment_base)
s1_result_base_25 = fine_tune(
    enc_learn           = s1_tuned_model_base_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.10it/s]


In [134]:
s1_tuned_model_base_25 = deepcopy(s1_result_base_25[7])
print(s1_tuned_model_base_25.__class__)
s1_time_base_25 = s1_result_base_25[4]
print("Train time: ", s1_time_base_25)
s1_loss_pre_base_25 = s1_result_base_25[1]['loss'][0]
print("Loss pre:", s1_loss_pre_base_25)
s1_loss_post_base_25 = s1_result_base_25[2]['loss'][0]
print("Loss post:", s1_loss_post_base_25)
s1_improvement_base_25 = (s1_loss_pre_base_25-s1_loss_post_base_25)*100/s1_loss_pre_base_25
print("Improvement: ", s1_improvement_base_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  5.11552619934082
Loss pre: 9.312607860565185
Loss post: 9.30456844329834
Improvement:  0.08632831304847073


In [97]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        tuned_model_base_25,
        f'moment-base-s1-25',
        type = 'learner'
    )
)

## Base 75

In [98]:
s1_tuned_model_base_75 = deepcopy(moment_base)
s1_result_base_75 = fine_tune(
    enc_learn           = s1_tuned_model_base_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:34,  1.46s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:10<00:00,  2.38it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.39it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.17it/s]


In [99]:
s1_tuned_model_base_75 = deepcopy(s1_result_base_75[7])
s1_time_base_75 = s1_result_base_75[4]
print("Train time: ", s1_time_base_75)
s1_loss_pre_base_75 = s1_result_base_75[1]['loss'][0]
print("Loss pre:", s1_loss_pre_base_75)
s1_loss_post_base_75 = s1_result_base_75[2]['loss'][0]
print("Loss post:", s1_loss_post_base_25)
s1_improvement_base_75 = (s1_loss_pre_base_75-s1_loss_post_base_75)*100/s1_loss_pre_base_75
print("Improvement: ", s1_improvement_base_75)

Train time:  5.034172058105469
Loss pre: 9.610819110870361
Loss post: 9.30456844329834
Improvement:  -0.42309067449458443


In [100]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        s1_tuned_model_base_75,
        f'moment-base-s1-75',
        type = 'learner'
    )
)

## Large 25

In [101]:
s1_tuned_model_large_25 = deepcopy(moment_large)
s1_result_large_25 = fine_tune(
    enc_learn           = s1_tuned_model_large_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.30it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.11it/s]


In [133]:
s1_tuned_model_large_25 = deepcopy(s1_result_large_25[7])
print(s1_tuned_model_large_25.__class__)
s1_time_large_25 = s1_result_large_25[4]
print("Train time: ", s1_time_large_25)
s1_loss_pre_large_25 = s1_result_large_25[1]['loss'][0]
print("Loss pre:", loss_pre_large_25)
s1_loss_post_large_25 = s1_result_large_25[2]['loss'][0]
print("Loss post:", s1_loss_post_large_25)
s1_improvement_large_25 = (s1_loss_pre_large_25-s1_loss_post_large_25)*100/s1_loss_pre_large_25
print("Improvement: ", s1_improvement_large_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  5.3530967235565186
Loss pre: 0.0018025103911188329
Loss post: 8.004377136230469
Improvement:  0.41313160400340676


## Large 75

In [103]:
s1_tuned_model_large_75 = deepcopy(moment_large)
s1_result_large_75 = fine_tune(
    enc_learn           = s1_tuned_model_large_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.25it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.24it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:12<00:00,  2.07it/s]


In [104]:
s1_tuned_model_large_75 = deepcopy(s1_result_large_75[7])
print(s1_tuned_model_large_75.__class__)
s1_time_large_75 = s1_result_large_75[4]
print("Train time: ", s1_time_large_75)
s1_loss_pre_large_75 = s1_result_large_75[1]['loss'][0]
print("Loss pre:", loss_pre_large_75)
s1_loss_post_large_75 = s1_result_large_75[2]['loss'][0]
print("Loss post:", s1_loss_post_large_75)
s1_improvement_large_75 = (s1_loss_pre_large_75-s1_loss_post_large_75)*100/s1_loss_pre_large_75
print("Improvement: ", s1_improvement_large_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  5.450057506561279
Loss pre: 0.003920885999428315
Loss post: 8.774216747283935
Improvement:  -0.031161936765054365


In [105]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        s1_tuned_model_large_75,
        f'moment-large-s1-75',
        type = 'learner'
    )
)

## Summary

In [135]:
print("--- Small 25 ---")
print("time: ", s1_time_small_25)
print("loss pre: ", s1_loss_pre_small_25)
print("loss_post: ", s1_loss_post_small_25)
print("improvement: ", s1_improvement_small_25)
print("--- Small 75 ---")
print("time: ", s1_time_small_75)
print("loss pre: ", s1_loss_pre_small_75)
print("loss_post: ", s1_loss_post_small_75)
print("improvement: ", s1_improvement_small_75)
print("--- Base 25 ---")
print("time: ", s1_time_base_25)
print("loss pre: ", s1_loss_pre_base_25)
print("loss_post: ", s1_loss_post_base_25)
print("improvement: ", s1_improvement_base_25)
print("--- Base 75 ---") 
print("time: ", s1_time_base_75)
print("loss pre: ", s1_loss_pre_base_75)
print("loss_post: ", s1_loss_post_base_75)
print("improvement: ", s1_improvement_base_75)
print("--- Large 25 ---") 
print("time: ", s1_time_large_25)
print("loss pre: ", s1_loss_pre_large_25)
print("loss_post: ", s1_loss_post_large_25)
print("improvement: ", s1_improvement_large_25)
print("--- Large 75 ---")
print("time: ", s1_time_large_75)
print("loss pre: ", s1_loss_pre_large_75)
print("loss_post: ", s1_loss_post_large_75)
print("improvement: ", s1_improvement_large_75)

--- Small 25 ---
time:  5.177350044250488
loss pre:  10.927521171569824
loss_post:  10.954853057861328
improvement:  -0.25011972854936104
--- Small 75 ---
time:  5.014077663421631
loss pre:  11.1762308883667
loss_post:  11.139793300628662
improvement:  0.3260275141234326
--- Base 25 ---
time:  5.11552619934082
loss pre:  9.312607860565185
loss_post:  9.30456844329834
improvement:  0.08632831304847073
--- Base 75 ---
time:  5.034172058105469
loss pre:  9.610819110870361
loss_post:  9.651481590270997
improvement:  -0.42309067449458443
--- Large 25 ---
time:  5.3530967235565186
loss pre:  8.037582931518555
loss_post:  8.004377136230469
improvement:  0.41313160400340676
--- Large 75 ---
time:  5.450057506561279
loss pre:  8.771483383178712
loss_post:  8.774216747283935
improvement:  -0.031161936765054365


# Generando los modelos para Toy

In [111]:
# Dataset
dataset = 'toy'
dataset_version = 'v2'
artifact_dataset = folder + dataset + ':' + dataset_version
print("Getting dataset artifact: ", artifact_dataset)
toy_artifact = wandb_api.artifact(artifact_dataset, type = 'dataset')
toy = toy_artifact.to_df()
print(toy.shape)
toy.head()

Getting dataset artifact:  mi-santamaria/deepvats/toy:v2


wandb:   1 of 1 files downloaded.  


(550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


In [112]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": 32,
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": None,
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": 1e-6,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": [30],
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 0,
    "print_to_path": True,
    "print_both": True,
    "print_path": "./generate-paper-models.log",
    "print_mode": "a",
    "use_moment_masks": True,
    "mask_stateful": True,
    "mask_future": False,
    "mask_sync": False,
    "analysis_mode": 'online',
    "use_wandb": True,
    "norm_by_sample": False,
    "norm_use_single_batch": False,
    "show_plot": True,
    "metrics": None,
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None,
    "criterion": torch.nn.MSELoss(),
    "training_percent": 0.25,
    "validation_percent": 0.3,
    "num_epochs" : 1, 
    "n_window_sizes": 1,
    "register_errors": False,
    "save_best_or_last": True
}

In [113]:
toy_tuned_model_small_25 = deepcopy(moment_small)
toy_result_small_25 = fine_tune(
    enc_learn           = toy_tuned_model_small_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.28it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.28it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.09it/s]


In [114]:
toy_tuned_model_small_25 = deepcopy(result[7])
toy_tuned_model_small_25.__class__
toy_time_small_25 = toy_result_small_25[4]
print("time: ", toy_time_small_25)
toy_loss_pre_small_25 = toy_result_small_25[1]['loss'][0]
print("loss pre: ", toy_loss_pre_small_25)
toy_loss_post_small_25 = toy_result_small_25[2]['loss'][0]
print("loss_post: ", toy_loss_post_small_25)
toy_improvement_small_25 = (toy_loss_pre_small_25-toy_loss_post_small_25)*100/toy_loss_pre_small_25
print(toy_improvement_small_25)

time:  5.5931079387664795
loss pre:  6.506192264556884
loss_post:  6.528851070404053
-0.3482652360368206


In [115]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        toy_tuned_model_small_25,
        f'moment-small-toy-25',
        type = 'learner'
    )
)

In [116]:
toy_tuned_model_small_75 = deepcopy(moment_small)
toy_result_small_75 = fine_tune(
    enc_learn           = toy_tuned_model_small_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:10<00:00,  2.40it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.40it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.15it/s]


In [117]:
toy_tuned_model_small_75 = deepcopy(toy_result_small_75[7])
print(toy_tuned_model_small_75.__class__)
toy_time_small_75 = toy_result_small_75[4]
print(toy_time_small_75)
toy_loss_pre_small_75 = toy_result_small_75[1]['loss'][0]
print(toy_loss_pre_small_75)
toy_loss_post_small_75 = toy_result_small_75[2]['loss'][0]
print(toy_loss_post_small_75)
toy_improvement_small_75 = (toy_loss_pre_small_75-toy_loss_post_small_75)*100/toy_loss_pre_small_75
print(toy_improvement_small_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
5.581114768981934
6.812604274749756
6.836425724029541
-0.3496672978361232


In [118]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        toy_tuned_model_small_75,
        f'moment-small-toy-75',
        type = 'learner'
    )
)

In [119]:
toy_tuned_model_base_25 = deepcopy(moment_base)
toy_result_base_25 = fine_tune(
    enc_learn           = toy_tuned_model_base_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:34,  1.44s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:12<00:00,  2.12it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:13<00:00,  2.13it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:13<00:00,  1.82it/s]


In [136]:
toy_tuned_model_base_25 = deepcopy(toy_result_base_25[7])
print(toy_tuned_model_base_25.__class__)
toy_time_base_25 = toy_result_base_25[4]
print("Train time: ", toy_time_base_25)
toy_loss_pre_base_25 = toy_result_base_25[1]['loss'][0]
print("Loss pre:", toy_loss_pre_base_25)
toy_loss_post_base_25 = toy_result_base_25[2]['loss'][0]
print("Loss post:", toy_loss_post_base_25)
toy_improvement_base_25 = (toy_loss_pre_base_25-toy_loss_post_base_25)*100/toy_loss_pre_base_25
print("Improvement: ", toy_improvement_base_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  5.577563762664795
Loss pre: 5.475051031112671
Loss post: 5.459954357147216
Improvement:  0.2757357672041048


In [121]:
toy_tuned_model_base_75 = deepcopy(moment_base)
toy_result_base_75 = fine_tune(
    enc_learn           = toy_tuned_model_base_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.46s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.29it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.29it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:11<00:00,  2.09it/s]


In [122]:
toy_tuned_model_base_75 = deepcopy(toy_result_base_75[7])
toy_time_base_75 = toy_result_base_75[4]
print("Train time: ", toy_time_base_75)
toy_loss_pre_base_75 = toy_result_base_75[1]['loss'][0]
print("Loss pre:", toy_loss_pre_base_75)
toy_loss_post_base_75 = toy_result_base_75[2]['loss'][0]
print("Loss post:", toy_loss_post_base_25)
toy_improvement_base_75 = (toy_loss_pre_base_75-toy_loss_post_base_75)*100/toy_loss_pre_base_75
print("Improvement: ", toy_improvement_base_75)

Train time:  5.977465391159058
Loss pre: 5.991312007904053
Loss post: 5.459954357147216
Improvement:  0.7239298845492788


In [123]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        toy_tuned_model_base_75,
        f'moment-base-toy-75',
        type = 'learner'
    )
)

In [124]:
toy_tuned_model_large_25 = deepcopy(moment_large)
toy_result_large_25 = fine_tune(
    enc_learn           = toy_tuned_model_large_25,
    window_mask_percent = 0.25,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.47s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.20it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:11<00:00,  2.20it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:12<00:00,  2.02it/s]


In [137]:
toy_tuned_model_large_25 = deepcopy(toy_result_large_25[7])
print(toy_tuned_model_large_25.__class__)
toy_time_large_25 = toy_result_large_25[4]
print("Train time: ", toy_time_large_25)
toy_loss_pre_large_25 = toy_result_large_25[1]['loss'][0]
print("Loss pre:", loss_pre_large_25)
toy_loss_post_large_25 = toy_result_large_25[2]['loss'][0]
print("Loss post:", toy_loss_post_large_25)
toy_improvement_large_25 = (toy_loss_pre_large_25-toy_loss_post_large_25)*100/toy_loss_pre_large_25
print("Improvement: ", toy_improvement_large_25)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  6.27472186088562
Loss pre: 0.0018025103911188329
Loss post: 4.929596614837647
Improvement:  -1.105741798988864


In [126]:
toy_tuned_model_large_75 = deepcopy(moment_large)
toy_result_large_75 = fine_tune(
    enc_learn           = toy_tuned_model_large_75,
    window_mask_percent = 0.75,
    verbose             = 8,
    **common_args
)

[8] [ --> _set_enc_input ][ --> _set_enc_input ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune ][ --> fine_tune ] | ./generate-paper-models.log
[8] [ --> set_fine_tune_ ][ --> set_fine_tune_ ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_single_ ][ --> fine_tune_moment_single_ ] | ./generate-paper-models.log
[8] [ --> set_train_and_eval_dataloaders ][ --> set_train_and_eval_dataloaders ] | ./generate-paper-models.log


  0% 0/25 [00:00<?, ?it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


  4% 1/25 [00:01<00:35,  1.46s/it]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


...

 92% 23/25 [00:11<00:00,  2.10it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


 96% 24/25 [00:12<00:00,  2.10it/s]

[8] [ --> moment_set_masks ][ --> moment_set_masks ] | ./generate-paper-models.log
[8] [ --> check_batch_masks ][ --> check_batch_masks ] | ./generate-paper-models.log
[8] [ --> check_mask ][ --> check_mask ] | ./generate-paper-models.log
[8] [ --> moment_safe_forward_pass ][ --> moment_safe_forward_pass ] | ./generate-paper-models.log
[8] [ --> moment_compute_loss ][ --> moment_compute_loss ] | ./generate-paper-models.log
[8] [ --> fine_tune_moment_eval_preprocess ][ --> fine_tune_moment_eval_preprocess ] | ./generate-paper-models.log


100% 25/25 [00:12<00:00,  2.00it/s]


In [127]:
toy_tuned_model_large_75 = deepcopy(toy_result_large_75[7])
print(toy_tuned_model_large_75.__class__)
toy_time_large_75 = toy_result_large_75[4]
print("Train time: ", toy_time_large_75)
toy_loss_pre_large_75 = toy_result_large_75[1]['loss'][0]
print("Loss pre:", loss_pre_large_75)
toy_loss_post_large_75 = toy_result_large_75[2]['loss'][0]
print("Loss post:", toy_loss_post_large_75)
toy_improvement_large_75 = (toy_loss_pre_large_75-toy_loss_post_large_75)*100/toy_loss_pre_large_75
print("Improvement: ", toy_improvement_large_75)

<class 'momentfm.models.moment.MOMENTPipeline'>
Train time:  6.074782848358154
Loss pre: 0.003920885999428315
Loss post: 5.635745782852172
Improvement:  -1.7047248734711387


In [128]:
# Log the model
run.log_artifact(
    ReferenceArtifact(
        toy_tuned_model_large_75,
        f'moment-large-toy-75',
        type = 'learner'
    )
)

In [139]:
print("--- Small 25 ---")
print("time: ", toy_time_small_25)
print("loss pre: ", toy_loss_pre_small_25)
print("loss_post: ", toy_loss_post_small_25)
print("improvement: ", toy_improvement_small_25)
print("--- Small 75 ---")
print("time: ", toy_time_small_75)
print("loss pre: ", toy_loss_pre_small_75)
print("loss_post: ", toy_loss_post_small_75)
print("improvement: ", toy_improvement_small_75)
print("--- Base 25 ---")
print("time: ", toy_time_base_25)
print("loss pre: ", toy_loss_pre_base_25)
print("loss_post: ", toy_loss_post_base_25)
print("improvement: ", toy_improvement_base_25)
print("--- Base 75 ---") 
print("time: ", toy_time_base_75)
print("loss pre: ", toy_loss_pre_base_75)
print("loss_post: ", toy_loss_post_base_75)
print("improvement: ", toy_improvement_base_75)
print("--- Large 25 ---") 
print("time: ", toy_time_large_25)
print("loss pre: ", toy_loss_pre_large_25)
print("loss_post: ", toy_loss_post_large_25)
print("improvement: ", toy_improvement_large_25)
print("--- Large 75 ---")
print("time: ", toy_time_large_75)
print("loss pre: ", toy_loss_pre_large_75)
print("loss_post: ", toy_loss_post_large_75)
print("improvement: ", toy_improvement_large_75)

--- Small 25 ---
time:  5.5931079387664795
loss pre:  6.506192264556884
loss_post:  6.528851070404053
improvement:  -0.3482652360368206
--- Small 75 ---
time:  5.581114768981934
loss pre:  6.812604274749756
loss_post:  6.836425724029541
improvement:  -0.3496672978361232
--- Base 25 ---
time:  5.577563762664795
loss pre:  5.475051031112671
loss_post:  5.459954357147216
improvement:  0.2757357672041048
--- Base 75 ---
time:  5.977465391159058
loss pre:  5.991312007904053
loss_post:  5.947939109802246
improvement:  0.7239298845492788
--- Large 25 ---
time:  6.27472186088562
loss pre:  4.8756841373443605
loss_post:  4.929596614837647
improvement:  -1.105741798988864
--- Large 75 ---
time:  6.074782848358154
loss pre:  5.541282167434693
loss_post:  5.635745782852172
improvement:  -1.7047248734711387
